## SLS Engine (Smart Location Search Engine)

*The SLS Engine (Smart Location Search Engine) is a sophisticated and advanced search engine designed specifically for finding places. It utilizes intelligent algorithms and data processing techniques to provide accurate and relevant results based on user queries.*

Here are some key features and characteristics of the SLS Engine:

- Intelligent Algorithm: The engine incorporates an intelligent algorithm that takes into account various factors such as location, user preferences, ratings, popularity, and other relevant data to deliver highly targeted search results.

- Geolocation Capabilities: The SLS Engine leverages geolocation data to identify the user's current location or any specified location, enabling more precise and personalized search results.

- Comprehensive Database: It maintains a comprehensive and up-to-date database of places, including restaurants, hotels, attractions, businesses, landmarks, and more. This ensures that users have access to a wide range of options when searching for places.

- Detailed Information: The engine provides detailed information about each place, including address, contact details, user ratings and reviews, photos, opening hours, and any additional relevant information to help users make informed decisions.

- Advanced Filtering Options: The SLS Engine offers advanced filtering options, allowing users to refine their search results based on specific criteria such as price range, cuisine type, amenities, ratings, and more.

- Predictive Search Suggestions: It incorporates predictive search suggestions, offering users real-time suggestions as they type their search queries, making the search process more efficient and seamless.

**Overall, the SLS Engine is a powerful and intelligent search tool that simplifies the process of finding places by providing accurate, personalized, and relevant results based on user preferences and location.**

Steps considered as part of the implementation:

- Importing shared FourSquare data from the Databricks Market Place.
- Creating a schema in our Unity Catalog.
- Creating a FourSquare Data table in our Own catalog based on the previous step.
- Transforming the imported columns from the given dataset.
- Creating a Vector Endpoint.
- Creating the final content column with attributes relevant to the search engine.
- Creating a Vector Index column from Step 6.
- Integrate a Text bot to generate smart results based on the implemented search engine.

In [0]:
%sql
create table workspace.default.places_us_nyc
as
select * from foursquare_places_free_new_york_city_sample;

In [0]:
%sql 
select * from workspace.default.places_us_nyc limit 5;

fsq_id,name,name_translated,latitude,longitude,geocodes,address,address_extended,locality,dma,region,postcode,country,admin_region,post_town,neighborhood,po_box,date_created,date_refreshed,fsq_category_ids,fsq_category_labels,fsq_chain_id,fsq_chain_name,parent_id,subvenue_count,hours,hours_popular,hours_display,tel,website,fax,email,facebook_id,instagram,twitter,description,rating,price,total_tips,tips,tastes,total_photos,photos,popularity,venue_reality_bucket,provenance_rating,date_closed,closed_bucket,atm,beer,barservice,businessmeeting,byo,clean,coatcheck,cocktails,crowded,datespopular,delivery,dressy,drivethrough,essentialreservations,familiespopular,fullbar,glutenfreediet,goodfordogs,groupsonlyreservations,groupspopular,hasmusic,hasparking,healthydiet,jukeboxmusic,latenight,livemusic,noisy,onlinereservations,outdoorseating,privatelot,privateroom,publiclot,quickbite,reservations,restroom,romantic,servesbarsnacks,servesbreakfast,servesbrunch,servesdessert,servesdinner,serveshappyhour,serveslunch,servestastingmenu,servicequality,singlespopular,sitdowndining,smoking,store_id,specialoccasion,streetparking,takeout,takesamex,takescreditcards,takesdinersclub,takesdiscover,takesmastercard,takesnfc,takesunionpay,takesvisa,trendy,tvs,valetparking,valueformoney,vegandiet,vegetariandiet,wheelchairaccessible,wifi,wine,census_block_id
23dbd876e604430510e9f69d,Bilkent University,null,40.748328,-73.984879,"Map(main -> Map(longitude -> -73.984879, latitude -> 40.748328), roof -> Map(longitude -> -73.984879, latitude -> 40.748328))",350 5th Ave,null,New York,New York,NY,10118,us,null,null,"List(East Side, Flatiron, Flatiron District, Garment District, Koreatown, Midtown, Midtown South, Midtown West, Murray Hill)",null,2012-05-21,2022-10-16,List(12013),"List(List(Community and Government, Education, College and University))",null,null,null,null,null,null,null,(212) 564-0175,http://www.bsa-gnyc.org,null,rpetrillo@bsa-gnyc.org,null,null,null,null,null,null,null,null,null,null,null,0.18,High,4,null,Unsure,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,360610076001001
4a22d7f9f964a520977d1fe3,Sant Ambroeus,null,40.775413,-73.962997,"Map(main -> Map(longitude -> -73.962997, latitude -> 40.775413), roof -> Map(longitude -> -73.962997, latitude -> 40.775413))",1000 Madison Ave,btwn 77th 78th St.,New York,null,NY,10075,us,null,null,"List(Upper East Side, Uptown)",null,2009-05-31,2023-05-16,List(13236),"List(List(Dining and Drinking, Restaurant, Italian Restaurant))",null,null,null,null,"Map(sunday -> List(List(8:00, 22:00)), tuesday -> List(List(8:00, 22:00)), wednesday -> List(List(8:00, 22:00)), monday -> List(List(8:00, 22:00)), friday -> List(List(8:00, 22:00)), thursday -> List(List(8:00, 22:00)), saturday -> List(List(8:00, 22:00)))","Map(sunday -> List(List(10:00, 19:00)), tuesday -> List(List(9:00, 11:00), List(13:00, 17:00), List(19:00, 20:00)), wednesday -> List(List(9:00, 19:00)), monday -> List(List(9:00, 18:00)), friday -> List(List(9:00, 18:00), List(21:00, 22:00)), thursday -> List(List(9:00, 18:00)), saturday -> List(List(9:00, 20:00)))",Open Daily 8:00 AM-10:00 PM,(212) 570-2211,http://www.santambroeus.com,(212) 570-2214,media@santambroeus.com,179539995429661,null,sant_ambroeus,null,8.78,VeryExpensive,90,"List(List(542c18c5498ead7dbea9041a, Lovely ambience. The coffee was good- more like a light Italian roast. We had a Napoleon and a custard and fruit tart, both of which were really pretty to look at and tasted amazing!), List(4e7cff04a17c03c4fc905526, Great ambiance and amazing staff. You can stop in for a delicious coffee or Gelato or have a posh meal – the Crab Salad is delicious as are ALL of the desserts.), List(515c7029e4b00a81e2c2b86

In [0]:
%sql 
create table `workspace`.`default`.`places_us_nyc_content` as 
select fsq_id ,
name, 
rating,
string (
  coalesce(address,"") || " " ||
  coalesce(address_extended,"") || " " ||
  coalesce(locality,"") || " " ||
  coalesce(region,"")  || " " ||
  coalesce(postcode,"") || " " ||
  coalesce(price,"") || " " || 
  coalesce(replace(replace (string(neighborhood), "]",""),"[",""),"") || " " ||
  coalesce(replace(replace (string(fsq_category_labels), "]",""),"[",""),"")  
) fsq_content
from `foursquare_places_free_new_york_city_sample`.`quickstart_schema`.`places_us_nyc` limit 10


In [0]:
%sql 
select * from default.places_us_nyc_content limit 5;

fsq_id,name,rating,fsq_content
559c39d7498e9b3ebf80af2f,Intexdesign,null,"252 W 38th St New York NY 10018 Garment District, Midtown, Midtown South, Midtown West, Theatre District Retail, Textiles Store"
d13291417888410f586e1d4b,Pan AM Equities,null,"65 4th Ave Frnt 1 New York NY 10003 Bowery, Downtown, East Village, NoHo Business and Professional Services, Real Estate Service, Real Estate Agency"
4a7e76f620224f68c14062bf,Fragments Showroom,null,"997 Madison Ave # 1 New York NY 10075 Upper East Side, Uptown Retail, Fashion Retail, Jewelry Store"
5a2079ff23472477f540ad59,Skillshare HQ,null,"407 Broome St New York NY 10013 Chinatown, Downtown, Little Italy, Lower Manhattan, Nolita, Soho Community and Government, Education"
a8e0597c623b417683907a9b,Metropolitan Community Investment Partners,null,"9 E 45th St New York NY 10017 Midtown, Midtown South, Midtown West, Murray Hill, Tenderloin, Theatre District Business and Professional Services, Computer Repair Service"



*This is foundational table, in the context of SLS Creation, refers to a core data table that serves as the basis for various data operations and analyses. It is typically created at the early stages of a project to establish a solid foundation for subsequent activities.*

##### Considered Sample Volume
*Given the limitation in computational resources, we have opted to utilize a sample dataset for demonstrating the functionality and viability of our working prototype.*

```create table workspace.default.places_us_nyc_content_sample_5K as select * from workspace.default.places_us_nyc_content limit 5000;```

##### Added Delta properties. 
*When creating a table for Vector Index creation, certain properties are necessary to ensure the effectiveness and efficiency of the indexing process. These properties define the characteristics of the table and its columns, allowing for optimized vector indexing. Some important table properties for Vector Index creation include:*

```ALTER TABLE `workspace`.`default`.`places_us_nyc_content_sample_5K` SET TBLPROPERTIES (delta.enableChangeDataFeed = true)```

##### Create Vearch serach endpoint 
*vectordb-ep*

##### Create Vearch serach Index 
*sample_content_ind*

####  install the required libraries 

In [0]:
%pip install mlflow==2.10.1 lxml==4.9.3 transformers==4.30.2 langchain==0.1.5 databricks-vectorsearch==0.22
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 108.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 89.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 38.8 MB/s eta 

In [0]:
import mlflow.deployments
deploy_client = mlflow.deployments.get_deploy_client("databricks")

from databricks.vector_search.client import VectorSearchClient
vsc = VectorSearchClient(disable_notice=True)

question = "List italian restaurants in midtown"

results = vsc.get_index("vectordb-ep", "workspace.default.sample_content_ind").similarity_search(
  query_text=question,
  columns=["fsq_id","name","rating","fsq_content"],
  num_results=5)
docs = results.get('result', {}).get('data_array', [])
docs

[['50bba642e4b03a9fdb698d43',
  'Le Grande Caffe',
  None,
  '1078 1st Ave  New York NY 10022 Moderate Midtown, Sutton Place, Turtle Bay Dining and Drinking, Restaurant, Pizzeria, Dining and Drinking, Restaurant, American Restaurant, Dining and Drinking, Restaurant, Italian Restaurant',
  0.6210756],
 ['061610be2f6e42db7792d095',
  'Allora Rock Center',
  None,
  '25 W 51st St  New York NY 10019  Midtown, Midtown West, San Juan Hill, Theatre District, West Side Dining and Drinking, Bar, Karaoke Bar, Dining and Drinking, Restaurant, Italian Restaurant',
  0.6187435],
 ['77054d9541b54d43214d8a10',
  'Amore Trattoria',
  None,
  '125 W 26th St  New York NY 10001  Chelsea, Flatiron, Flatiron District, Garment District, Midtown, Midtown South Dining and Drinking, Restaurant, Italian Restaurant',
  0.6170077],
 ['4d4e0cc51a2d721e586e9e1d',
  "New York Sal's Pizza",
  6.41,
  "696 10th Ave  New York NY 10019 Cheap Clinton, Hell's Kitchen, Midtown, Midtown West, Theatre District, West Side Din

In [0]:
import pandas as pd
results=pd.DataFrame(docs)
results.columns=["fsq_id","name","rating","fsq_content",'match_score']
results=results.fillna(0)


##### Rank Results by Vector Match

In [0]:
results.sort_values('match_score',ascending=False)

,fsq_id,name,rating,fsq_content,match_score
0,50bba642e4b03a9fdb698d43,Le Grande Caffe,0.00,1078 1st Ave New York NY 10022 Moderate Midto...,0.621076
1,061610be2f6e42db7792d095,Allora Rock Center,0.00,"25 W 51st St New York NY 10019 Midtown, Midt...",0.618744
2,77054d9541b54d43214d8a10,Amore Trattoria,0.00,"125 W 26th St New York NY 10001 Chelsea, Fla...",0.617008
3,4d4e0cc51a2d721e586e9e1d,New York Sal's Pizza,6.41,"696 10th Ave New York NY 10019 Cheap Clinton,...",0.602478
4,4b0e0b45f964a520545423e3,Morton's The Steakhouse,7.71,551 5th Ave New York NY 10176 Expensive Midto...,0.600040


##### Rank Results by Rating

In [0]:
results.sort_values('rating',ascending=False)

,fsq_id,name,rating,fsq_content,match_score
4,4b0e0b45f964a520545423e3,Morton's The Steakhouse,7.71,551 5th Ave New York NY 10176 Expensive Midto...,0.600040
3,4d4e0cc51a2d721e586e9e1d,New York Sal's Pizza,6.41,"696 10th Ave New York NY 10019 Cheap Clinton,...",0.602478
0,50bba642e4b03a9fdb698d43,Le Grande Caffe,0.00,1078 1st Ave New York NY 10022 Moderate Midto...,0.621076
1,061610be2f6e42db7792d095,Allora Rock Center,0.00,"25 W 51st St New York NY 10019 Midtown, Midt...",0.618744
2,77054d9541b54d43214d8a10,Amore Trattoria,0.00,"125 W 26th St New York NY 10001 Chelsea, Fla...",0.617008
